In [35]:
import boto3

ACCESS_KEY = "Add your AWS Access Key"
SECRET_KEY = "Add your AWS secret key"
REGION = "us-east-2"
BUCKET = "prueba-tecnica-juan-cano"
PATH_LOCAL = r"C:\Users\CaJu670\Desktop\prueba-wenia-juan-cano\direcciones_originales.csv"
NOMBRE_REMOTO = "variantes_direcciones.csv"

s3 = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=REGION
)

s3.upload_file(PATH_LOCAL, BUCKET, NOMBRE_REMOTO)
print("Archivo subido correctamente a S3")


Archivo subido correctamente a S3


In [ ]:
import pandas as pd

# direcciones_originales.csv, fue un archivo que tuve que crear yo mismo porque no lo recibí con la prueba técnica 
df = pd.read_csv('direcciones_originales.csv')

def generar_variantes(direccion):
    variantes = set()
    direccion = direccion.upper()
    variantes_generadas = [
        direccion.replace('CALLE', 'CLL'),
        direccion.replace('CARRERA', 'CRA'),
        direccion.replace('KRA', 'CRA'),
        direccion.replace('NRO', '#'),
        direccion.replace('NUM', "#"),
        direccion.replace('NUMERO', '#'),
    ]
    
    for variante in variantes_generadas:
        if variante != direccion:
            variantes.add(variante)
    return list(variantes)



In [37]:
rows = []
for original in df['direccion']:
    variants = generar_variantes(original)
    for variante in variants:
        rows.append({'original': original, 'variante': variante})
df_variantes = pd.DataFrame(rows)

#  El archivo direcciones_homonimas.csv contiene las variantes formas alternativas de escritura de cada dirección original
df_variantes.to_csv('direcciones_homonimas.csv', index=False)


similares = []
for _, row in df_variantes.iterrows():
    score = fuzz.token_sort_ratio(row['original'], row['variante'])
    if score > 90:
        similares.append({
            'original': row['original'],
            'variante': row['variante'],
            'similitud': score
        })

df_similares = pd.DataFrame(similares)

# El archivo direcciones_similares.csv es el resultado de comparar cada dirección original con sus variantes las del archivo de homónimas y quedarse solo con las variantes cuya similitud es mayor al 90%
df_similares.to_csv('direcciones_similares.csv', index=False)

In [38]:
import pandas as pd
import requests
import time

# Tu API Key de Google
GOOGLE_API_KEY = 'Add your google API KEY'

# Cargar direcciones similares
df = pd.read_csv('direcciones_similares.csv')

# Crear columnas vacías para latitud y longitud
df['lat'] = None
df['lng'] = None

for idx, row in df.iterrows():
    direccion = row['variante']
    direccion_query = f"{direccion}, Medellin, Colombia"  # Puedes adaptar la ciudad
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={direccion_query}&key={GOOGLE_API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['results']:
            location = result['results'][0]['geometry']['location']
            df.at[idx, 'lat'] = location['lat']
            df.at[idx, 'lng'] = location['lng']
    time.sleep(0.15)  


df.to_csv('direcciones_georreferenciadas.csv', index=False)


In [ ]:
import folium

# El archivo direcciones_georreferenciadas.csv contiene las coordenadas de las direcciones que cumplen con los parametros
df = pd.read_csv('direcciones_georreferenciadas.csv')


mapa = folium.Map(location=[6.2442, -75.5812], zoom_start=12)


for _, row in df.iterrows():
    lat, lng, direccion = row['lat'], row['lng'], row['variante']
    if pd.notnull(lat) and pd.notnull(lng):
        folium.Marker(
            [lat, lng],
            popup=direccion,
            icon=folium.Icon(color="blue", icon="info-sign"),
        ).add_to(mapa)

# Se guarda el mapa como HTML interactivo
mapa.save('mapa_direcciones.html')
